In [0]:
import os
import pandas as pd
import numpy as np
import nltk

from google.colab import drive
drive.mount('/content/gdrive')
path = 'gdrive/My Drive/AML/DataSet/'
os.listdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


['glove_embedding.txt', 'spam_detection.csv']

In [0]:
df = pd.read_csv(path + "spam_detection.csv", encoding = 'latin-1')
df = df.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
df.columns = ["Label", "Text"]
Map = {'ham': 0, 'spam': 1}
df["y"] = df["Label"].map(Map).astype(np.int64)
df.tail()

,Label,Text,y
5566,spam,This is the 2nd time we have tried 2 contact u...,1
5567,ham,Will Ì_ b going to esplanade fr home?,0
5568,ham,"Pity, * was in mood for that. So...any other s...",0
5569,ham,The guy did some bitching but I acted like i'd...,0
5570,ham,Rofl. Its true to its name,0


In [0]:
nltk.download('punkt')

texts = df["Text"].to_list()
texts = [text.lower() for text in texts]            # chuyển các đoạn text thành chữ thường (word embedding chỉ cho chữ thường)
tokenized_texts = [nltk.tokenize.word_tokenize(text) for text in texts]    # tách câu thành một list các từ

print(tokenized_texts[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['ok', 'lar', '...', 'joking', 'wif', 'u', 'oni', '...']


In [0]:
import io
import numpy as np
def load_word_embeddings(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    vocab, matrix = [], []
    
    for line in fin:
        tokens = line.rstrip().split(' ')
        vocab.append(tokens[0])
        matrix.append(list(map(float, tokens[1:])))
    return vocab, np.asarray(matrix)

In [0]:
vocab, matrix = load_word_embeddings(path + "glove_embedding.txt")

In [0]:
## Gán các mã
__PADDED_INDEX__ = 0    # mã dùng cho các vị trí chỉ có tính nối dài cho cùng kích thước
__UNKNOWN_WORD__ = 1    # mã cho những từ không có trong embedding

In [0]:
# Tạo một dictionary, có nhiệm vụ là một ánh xạ từ ảnh sang mã số,
# mã số được bắt đầu từ 2 vì số 0 và 1 được dành cho trường hợp đặc biệt
word_to_index = {word: index+2 for index, word in enumerate(vocab)}
len(word_to_index)

7342

In [0]:
# Do mã số được bắt đầu từ 2, nên cần thêm 2 vector vào đàu ma trận
embedding_matrix = np.pad(matrix, [[2,0],[0,0]], mode='constant', constant_values =0.0)
print(embedding_matrix.shape)
print(embedding_matrix)

# Khi đó, __PADDED_INDEX__ dùng dòng đầu tiên của  embedding_matrix
# __UNKNOWN_WORD__ dùng dòng thứ hai của embedding_matrix

(7362, 300)
[[ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.18378   -0.12123   -0.11987   ... -0.039038   0.18274    0.14654  ]
 ...
 [ 0.18754   -0.040832   0.19611   ...  0.079996   0.016479   0.17797  ]
 [-0.1167     0.0073071 -0.19401   ...  0.21549    0.015527   0.14948  ]
 [ 0.075198  -0.097452  -0.14505   ...  0.23842   -0.39966    0.14454  ]]


In [0]:
import tensorflow as tf
print(tf.__version__)

X = []
for text in tokenized_texts:
    cur_text_indices = []
    for word in text:
        if word in word_to_index:
            cur_text_indices.append(word_to_index[word])    ## map từ word sang index
        else:
            cur_text_indices.append(__UNKNOWN_WORD__)       ## gán unknown cho từ không có trong bộ glove
    X.append(cur_text_indices)

## pad data cho có cùng chiều dài
X = tf.keras.preprocessing.sequence.pad_sequences(sequences=X,       # sequences: list các câu có độ dài không bằng nhau
                                                  padding='post')    # vị trí pad là 'pre' (trước) hoặc 'post' (sau)

y = df['y'].values   ## Label của bài toán, 0 là không phải spam, 1 là spam
X.shape, y.shape

1.15.0


((5571, 215), (5571,))

In [0]:
## Chia data
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, random_state = 42)

In [0]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(None,))                   ## None biểu thị kích thước không xác định của câu

embed = Embedding(input_dim=embedding_matrix.shape[0],    ## Khai báo kích thước của vocab
                  output_dim=embedding_matrix.shape[1],   ## Khai báo kích thước của embedding
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable=False,                        ## Không cần thiết train embedding
                  mask_zero=True)(inputs)                 ## zero_mask: những vị trí có giá trị 0 không được tính toán, vì đó là giá trị thêm vào cho đủ độ dài mà thôi
                                                          ##  (__PADDED_INDEX__ gán bằng 0)

lstm = Bidirectional(LSTM(units=100,                      ## units: kích thước của hidden_state trong LSTM
            return_sequences=False))(embed)               ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng

dense = Dense(units=2, activation='softmax')(lstm)

model = Model(inputs=inputs,
              outputs=dense)

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# dense1 = Dense(1, activation='sigmoid')(lstm)
# model.compile(optimizer='rmsprop',
#              loss='binary_crossentropy',
#              metrics=['accuracy'])

model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_7 (Embedding)      (None, None, 300)         2208600   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 200)               320800    
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 402       
Total params: 2,529,802
Trainable params: 321,202
Non-trainable params: 2,208,600
_________________________________________________________________


In [0]:
# Checkpoint Callback
mc = tf.keras.callbacks.ModelCheckpoint(filepath="lstm_spam.h5", 
                                        monitor='val_loss',
                                        mode='min', 
                                        verbose=0, 
                                        save_best_only=True)
# Train
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
         epochs=10,
         callbacks=[mc])

model.load_weights("lstm_spam.h5")
_, val_acc = model.evaluate(X_valid, y_valid)
print("Accuracy on valid: ", val_acc)

Train on 4456 samples, validate on 1115 samples
Epoch 1/10
4456/4456 [==============================] - 124s 28ms/sample - loss: 0.1219 - acc: 0.9542 - val_loss: 0.1026 - val_acc: 0.9641
Epoch 2/10
4456/4456 [==============================] - 121s 27ms/sample - loss: 0.0483 - acc: 0.9852 - val_loss: 0.0456 - val_acc: 0.9839
Epoch 3/10
4456/4456 [==============================] - 120s 27ms/sample - loss: 0.0277 - acc: 0.9910 - val_loss: 0.0450 - val_acc: 0.9848
Epoch 4/10
4456/4456 [==============================] - 119s 27ms/sample - loss: 0.0180 - acc: 0.9942 - val_loss: 0.0595 - val_acc: 0.9812
Epoch 5/10
4456/4456 [==============================] - 119s 27ms/sample - loss: 0.0132 - acc: 0.9964 - val_loss: 0.0567 - val_acc: 0.9848
Epoch 6/10
4456/4456 [==============================] - 119s 27ms/sample - loss: 0.0054 - acc: 0.9991 - val_loss: 0.0495 - val_acc: 0.9848
Epoch 7/10
4456/4456 [==============================] - 118s 27ms/sample - loss: 0.0032 - acc: 0.9996 - val_loss: 0.05

In [0]:
def model_predict(texts):
  tests = [text.lower() for text in texts]            # chuyển các đoạn text thành chữ thường (word embedding chỉ cho chữ thường)
  tokenized_test = [nltk.tokenize.word_tokenize(test) for test in tests]    # tách câu thành một list các từ
  X_test = []
  for text in tokenized_test:
    cur_text_indices = []
    for word in text:
        if word in word_to_index:
            cur_text_indices.append(word_to_index[word])    ## map từ word sang index
        else:
            cur_text_indices.append(__UNKNOWN_WORD__)       ## gán unknown cho từ không có trong bộ glove
    X_test.append(cur_text_indices)

  ## pad data cho có cùng chiều dài
  X_test = tf.keras.preprocessing.sequence.pad_sequences(sequences=X_test,       # sequences: list các câu có độ dài không bằng nhau
                                                       padding='post')    # vị trí pad là 'pre' (trước) hoặc 'post' (sau)


  model.load_weights("lstm_spam.h5")
  prediction = model.predict(X_test)
  return prediction

In [0]:
tests=["wanna ask something? just send me a mess",
       "Urgent! You have won our competition's prize!! Please call us now.",
       "Call me to get a free holiday now"]
A = model_predict(tests)
result = np.argmax(A, axis = 1)
result

array([0, 1, 0])